In [1]:
import urllib.parse as urlp
import re
url = 'https://www.google.com/search?client=firefox-b-1-d&q=search'
urls = ['https://www.google.com/search?client=firefox-b-1-d&q=search','https://twitter.com/BarackObama?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor']
domain = urlp.urlparse(url).netloc
re.sub('^www\.','',domain)

def extract_domains(urls):
    ret = []
    for url in urls:
        domain = re.sub('^www\.','',urlp.urlparse(url).netloc)
        ret.append(domain)
    return ret

extract_domains(urls)

['google.com', 'twitter.com']

In [125]:
import os
os.chdir('/Users/dhek/Desktop/covid_gov_tweets/')

import tweepy
import keyring
import pandas as pd
import numpy as np
import string 
import time
from sqlalchemy import create_engine
import psycopg2

# import helpers as pj

# Helper functions 
# from helpers import import_tweets_to_db
# from helpers import get_last_tweet_id
# from helpers import loop_tweets
exec(open('./Python/helpers.py').read())


# Pull in Passwords
exec(open('./consumerpy').read())

# Open the Connection to Twitter
auth = tweepy.OAuthHandler(keyring.get_password('cov_gov','TWITTER_KEY'),keyring.get_password('cov_gov','TWITTER_SECRET'))
auth.set_access_token(keyring.get_password('cov_gov','TWITTER_TOKEN'), keyring.get_password('cov_gov','TWITTER_TOKEN_SEC'))
api = tweepy.API(auth,wait_on_rate_limit=True,retry_count=20,retry_delay=10,retry_errors=[401, 404, 500, 503],wait_on_rate_limit_notify=True)

screen_name = 'senatemajldr'
# get_oldest_tweet_id(screen_name= screen_name,db_str='postgres://127.0.0.1:5432/tt')
# loop_tweets(screen_name=screen_name, api= api, db_str = 'postgres://127.0.0.1:5432/tt')


In [2]:
import pandas as pd
import numpy as np
import string 
import psycopg2
from sqlalchemy import create_engine

## pull data from db
db_str = 'postgres://127.0.0.1:5432/tt'
obama = pd.read_sql("select * from tweets where tweeter = 'BarackObama'",create_engine(db_str))
obama = obama.assign(covid_and_retweet = ((obama.covid_related) & (obama.isRetweet)))
obama = obama.assign(not_covid_and_retweet = ((obama.covid_related==False) & (obama.isRetweet)))
sum_obama = (obama.groupby(['tweeter']).agg(
{'id':'count',
'isRetweet':'sum',
'covid_related':'sum',
'covid_and_retweet':'sum'}))

sum_obama.rename(columns = \
{'id' : 'Count of Tweets', \
'covid_related': 'Number COVID-Related', \
'isRetweet': 'Is Retweet', 'covid_and_retweet': 'COVID Retweet'})

obama[obama.not_covid_and_retweet]

,id,text,dttm,isRetweet,tweeter,covid_related,covid_and_retweet,not_covid_and_retweet
2,1296239491611590657,RT @DemConvention: .@BarackObama understands t...,2020-08-20 00:15:52,True,BarackObama,False,False,True
8,1295484440215420928,RT @DemConvention: You won’t want to miss @Mic...,2020-08-17 22:15:34,True,BarackObama,False,False,True
12,1293548976541663233,RT @KamalaHarris: We are in a battle for the s...,2020-08-12 14:04:43,True,BarackObama,False,False,True
49,1276654351109500932,RT @MichelleObama: Thinking back to that day r...,2020-06-26 23:11:31,True,BarackObama,False,False,True
50,1276568419979141121,RT @ObamaFoundation: In the afternoon of June ...,2020-06-26 17:30:03,True,BarackObama,False,False,True
...,...,...,...,...,...,...,...,...
3162,539537825872355328,RT @kerrywashington: #GetCovered Sign up for q...,2014-12-01 21:53:36,True,BarackObama,False,False,True
3164,539501426414608384,RT @VP: Affordable health care won't break the...,2014-12-01 19:28:58,True,BarackObama,False,False,True
3173,538080509629583360,RT @WhiteHouse: Today President Obama called U...,2014-11-27 21:22:45,True,BarackObama,False,False,True
3176,537701098598191104,RT @EnvAm: #ClimateThanks to the millions of A...,2014-11-26 20:15:06,True,BarackObama,False,False,True


In [142]:
import pandas as pd
import numpy as np
import string 
import psycopg2
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

## pull data from db
db_str = 'postgres://127.0.0.1:5432/tt'
t = pd.read_sql("select * from tweets",create_engine(db_str))
t['dttm'] = pd.to_datetime(t['dttm'])
t['week_of'] = t.apply(lambda x: x['dttm'].date() - timedelta(days=x['dttm'].weekday()), axis = 1)

g = (t[t['week_of'] >= datetime(2020,3,1).date()]
         .groupby(['tweeter','week_of'], as_index=False)
         .agg({'covid_related':'sum','id':'count'}))
g = g.rename(columns={'id':'tweets'})
g = g.assign(pct_covid = g.covid_related / g.tweets)
g['week_of'] = pd.to_datetime(g['week_of'])
g

,tweeter,week_of,covid_related,tweets,pct_covid
0,BarackObama,2020-03-02,1.0,4,0.250000
1,BarackObama,2020-03-09,0.0,5,0.000000
2,BarackObama,2020-03-16,0.0,12,0.000000
3,BarackObama,2020-03-23,0.0,10,0.000000
4,BarackObama,2020-03-30,1.0,3,0.333333
...,...,...,...,...,...
96,senatemajldr,2020-07-27,4.0,17,0.235294
97,senatemajldr,2020-08-03,1.0,15,0.066667
98,senatemajldr,2020-08-10,2.0,11,0.181818
99,senatemajldr,2020-08-17,0.0,5,0.000000


In [149]:
import altair as alt
from vega_datasets import data

selection = alt.selection_multi(fields=['tweeter'], bind='legend')
alt.Chart(g).mark_bar().encode(
    alt.X('week_of:T',axis=alt.Axis(format='%B', domain=False, tickSize=0)),
    alt.Y('tweets:Q', axis=None),
    alt.Color('tweeter:N'),
    opacity = alt.condition(selection, alt.value(1), alt.value(0.2))
).add_selection(
    selection
)



alt.Chart(...)

In [144]:
import os
os.chdir('/Users/dhek/Desktop/covid_gov_tweets/')

import tweepy
import keyring

# Pull in Passwords
exec(open('./consumerpy').read())

# Open the Connection to Twitter
auth = tweepy.OAuthHandler(keyring.get_password('cov_gov','TWITTER_KEY'),keyring.get_password('cov_gov','TWITTER_SECRET'))
auth.set_access_token(keyring.get_password('cov_gov','TWITTER_TOKEN'), keyring.get_password('cov_gov','TWITTER_TOKEN_SEC'))
api = tweepy.API(auth,wait_on_rate_limit=True,retry_count=20,retry_delay=10,retry_errors=[401, 404, 500, 503],wait_on_rate_limit_notify=True)

def get_full_tweet_text(status):
    if hasattr(status, "retweeted_status"):  # Check if Retweet
        try:
            return status.retweeted_status.full_text
        except AttributeError:
            return status.text
    else:
        try:
            return status.full_text
        except AttributeError:
            return status.text

username = 'realDonaldTrump'
user = api.get_user(username)
tweets = api.user_timeline(screen_name = user.screen_name, count=5, tweet_mode = 'extended')
for t in tweets:
    print(get_full_tweet_text(t))
    print('')


Just had a wonderful conversation with my friend, Prime Minister @AbeShinzo of Japan, who will be leaving office soon. Shinzo will soon be recognized as the greatest Prime Minister in the history of Japan, whose relationship with the USA is the best it has ever been. Special man!

When is Slow Joe Biden going to criticize the Anarchists, Thugs &amp; Agitators in ANTIFA? When is he going to suggest bringing up the National Guard in BADLY RUN &amp; Crime Infested Democrat Cities &amp; States? Remember, he can’t lose the Crazy Bernie Super Liberal vote!

....His problem is interesting. He must always be weak on CRIME because of the Bernie Sanders Radical Left voter. If he loses them, like Crooked Hillary did, he is “toast”, and many will vote for me because of TRADE (Bernie was good on trade). Joe MUST always be weak on crime!

Joe Biden is coming out of the basement earlier than his hoped for ten days because his people told him he has no choice, his poll numbers are PLUNGING! Going to P